> ## Neste notebook o usuário deve entrar com o nome da lei, o inciso e a alínea, e o programa retornará cada um dos parâmetros solicitados.

In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json

In [2]:
#simulando o click
url = 'https://www2.camara.leg.br/busca/?q=lei 9503'
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url)
time.sleep(10)
driver.find_element_by_xpath("//div[@id='resultadoBusca']//ul//li//span//a").click()
time.sleep(10)
driver.find_element_by_xpath("//div[@class='sessao']//a").click()
time.sleep(10)
element = driver.find_element_by_xpath("//div[@class='texto']")
html_content = element.get_attribute('outerHTML')
time.sleep(5)
driver.quit()

In [4]:
print(html_content)

In [6]:
#parseando o conteúdo
soup = BeautifulSoup(html_content, 'lxml')
#coletando o texto
text = soup.find_all(text=True)
type(text)#lista

bs4.element.ResultSet

In [ ]:
#removendo caracteres indesejáveis
for i in range(len(text)):
    text[i] = text[i].replace('\n', '')
    text[i] = text[i].replace('\t', '')
    text[i] = text[i].replace('\xa0', '')
#print(text)

In [8]:
#removendo string vazia da lista
lista_indices = []
for i in range(len(text)):
    if text[i] == '':
        lista_indices.append(i)
print(lista_indices)

In [9]:
for i in range(len(text)):
    if '' in text:
        text.remove('')

In [10]:
'' in text

False

> Até aqui, consegui todo o conteúdo do texto em uma lista. Então posso criar funções para varrer a lista atrás de caracteres que delimitem o início e o fim de um artigo, e seus possíveis incisos.

>Jogar os artigos em dicionarios, assim como seus incisos.

In [11]:
text

['O PRESIDENTE DA REPÚBLICA ',
 'Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei: ',
 'CAPÍTULO I',
 'DISPOSIÇÕES PRELIMINARES',
 'Art. 1º. O trânsito de qualquer natureza nas vias terrestres do território nacional, abertas à circulação, rege-se por este Código. ',
 '§ 1º Considera-se trânsito a utilização das vias por pessoas, veículos e animais, isolados ou em grupos, conduzidos ou não, para fins de circulação, parada, estacionamento e operação de carga ou descarga. ',
 '§ 2º O trânsito, em condições seguras, é um direito de todos e dever dos órgãos e entidades componentes do Sistema Nacional de Trânsito, a estes cabendo, no âmbito das respectivas competências, adotar as medidas destinadas a assegurar esse direito. ',
 '§ 3º Os órgãos e entidades componentes do Sistema Nacional de Trânsito respondem, no âmbito das respectivas competências, objetivamente, por danos causados aos cidadãos em virtude de ação, omissão ou erro na execução e manutenção de programas,

In [12]:
for i in range(len(text)):
    if 'Art. 1º.' in text[i]:
        print(text[i])
    if '§' in text[i]:
        print(text[i])
    if 'Art. 2º' in text[i]:
        break

Art. 1º. O trânsito de qualquer natureza nas vias terrestres do território nacional, abertas à circulação, rege-se por este Código. 
§ 1º Considera-se trânsito a utilização das vias por pessoas, veículos e animais, isolados ou em grupos, conduzidos ou não, para fins de circulação, parada, estacionamento e operação de carga ou descarga. 
§ 2º O trânsito, em condições seguras, é um direito de todos e dever dos órgãos e entidades componentes do Sistema Nacional de Trânsito, a estes cabendo, no âmbito das respectivas competências, adotar as medidas destinadas a assegurar esse direito. 
§ 3º Os órgãos e entidades componentes do Sistema Nacional de Trânsito respondem, no âmbito das respectivas competências, objetivamente, por danos causados aos cidadãos em virtude de ação, omissão ou erro na execução e manutenção de programas, projetos e serviços que garantam o exercício do direito do trânsito seguro. 
§ 4º (VETADO) 
§ 5º Os órgãos e entidades de trânsito pertencentes no Sistema Nacional de 

In [30]:
#trás todos os parágrafos do artigo, seja ele artigo de parágrafo único ou não
#n é o número do artigo
n = 2
m = 0
art = str('Art. ' + str(n))
for i in range(len(text)):
    if art in text[i]:
        print(text[i])
        m += 1
    if m != 0 and '§' in text[i]:
        print(text[i])
    if m != 0 and 'Parágrafo único.' in text[i]:
        print(text[i+1])
    if str('Art. ' + str(n+1)) in text[i]:
        break

Art. 2º. São vias terrestres urbanas e rurais as ruas, as avenidas, os logradouros, os caminhos, as passagens, as estradas e as rodovias, que terão seu uso regulamentado pelo órgão ou entidade com circunscrição sobre elas, de acordo com as peculiaridades locais e as circunstâncias especiais. 
Para os efeitos deste Código, são consideradas vias terrestres as praias abertas à circulação publica e as vias internas pertencentes aos condomínios constituídos por unidades autônomas. 


In [34]:
#tratando o inciso
n = 6
m = 0
inciso = 'II'
art = str('Art. ' + str(n))
for i in range(len(text)):
    if art in text[i]:
        print(text[i])
        m += 1
    if m != 0 and '§' in text[i]:
        print(text[i])
    if m != 0 and 'Parágrafo único.' in text[i]:
        print(text[i+1])
    if m != 0 and inciso in text[i]:
        print(text[i])
        m = 0# m retorna a zero para não trazer o próximo inciso
    if str('Art. ' + str(n+1)) in text[i]:
        break

Art. 6º. São objetivos básicos do Sistema Nacional de Trânsito: 
II - fixar, mediante normas e procedimentos, a padronização de critérios técnicos, financeiros e administrativos para a execução das atividades de trânsito; 


In [59]:
#tratando a alínea
n = 29
m = 0
inciso = 'III'
alinea = 'a)'
a = 0
an = 0
art = str('Art. ' + str(n))
for i in range(len(text)):
    if art in text[i]:
        print(text[i])
        m += 1
    if m != 0 and '§' in text[i]:
        print(text[i])
        
    if m != 0 and 'Parágrafo único.' in text[i]:
        print(text[i+1])
    if m != 0 and inciso in text[i]:
        print('Inciso' + text[i])
        m = 0# m retorna a zero para não trazer o próximo inciso
        a += 1 
    if a != 0 and an == 0 and alinea != '' and alinea in text[i]:
        print('Alinea '+ alinea + text[i+1])
        an += 1#para não trazer outra alínea
    if str('Art. ' + str(n+1)) in text[i]:
        break

Art. 29. O trânsito de veículos nas vias terrestres abertas a circulação obedecerá às seguintes normas: 
Inciso III - quando veículos, transitando por fluxos que se cruzem, se aproximarem de local não sinalizado, terá preferência de passagem: 
Alinea a)no caso de apenas um fluxo ser proveniente de rodovia, aquele que estiver circulando por ela; 


In [73]:
#tratando artigos que fogem a estrutura
n = 6
m = 0
inciso = 'III'
alinea = 'a)'
a = 0
an = 0
at = 0
art = str('Art. ' + str(n))
for i in range(len(text)):
    if art in text[i]:
        print(text[i])
        m += 1
    if m != 0 and '§' in text[i]:
        print(text[i])
        
    if m != 0 and 'Parágrafo único.' in text[i]:
        print(text[i+1])
        
    if m != 0 and inciso in text[i]:
        print('Inciso' + text[i])
        m = 0# m retorna a zero para não trazer o próximo inciso
        a += 1
     #--ler parágrafos--# 
        for j in range(i, len(text)):
            if m == 0 and '§' in text[j]:
                print(text[j])
            if str('Art. ' + str(n+1)) in text[j]:
                break
        
    if a != 0 and an == 0 and alinea != '' and alinea in text[i]:
        print('Alinea '+ alinea + text[i+1])
        an += 1#para não trazer outra alínea
        
    if str('Art. ' + str(n+1)) in text[i]:
        break
        
#o problema consiste em trazer os parágrafos de artigos desestruturados

Art. 6º. São objetivos básicos do Sistema Nacional de Trânsito: 
IncisoIII - estabelecer a sistemática de fluxos permanentes de informações entre os seus diversos órgãos e entidades, a fim de facilitar o processo decisório e a integração do Sistema. 
